<a href="https://colab.research.google.com/github/jessiececilya/Projects/blob/main/Movie_recommender_(content_Based).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [251]:
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import re
import string
pd.set_option('display.max_colwidth',100)

df = pd.read_csv('https://query.data.world/s/uikepcpffyo2nhig52xxeevdialfl7')

df = df[['Title','Genre','Director','Actors','Plot']]
df.head()

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, William Sadler","Two imprisoned men bond over a number of years, finding solace and eventual redemption through a..."
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard S. Castellano",The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert De Niro","The early life and career of Vito Corleone in 1920s New York is portrayed while his son, Michael..."
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine","When the menace known as the Joker emerges from his mysterious past, he wreaks havoc and chaos o..."
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G. Marshall",A jury holdout attempts to prevent a miscarriage of justice by forcing his colleagues to reconsi...


In [252]:
pip install rake_nltk

In [255]:
df.iloc[2]['Plot']

'The early life and career of Vito Corleone in 1920s New York is portrayed while his son, Michael, expands and tightens his grip on the family crime syndicate.'

In [258]:
for index, row in df.iterrows():
    # plot = row['Plot']
    print(row['Plot'])

Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.
The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.
The early life and career of Vito Corleone in 1920s New York is portrayed while his son, Michael, expands and tightens his grip on the family crime syndicate.
When the menace known as the Joker emerges from his mysterious past, he wreaks havoc and chaos on the people of Gotham, the Dark Knight must accept one of the greatest psychological and physical tests of his ability to fight injustice.
A jury holdout attempts to prevent a miscarriage of justice by forcing his colleagues to reconsider the evidence.
In German-occupied Poland during World War II, Oskar Schindler gradually becomes concerned for his Jewish workforce after witnessing their persecution by the Nazi Germans.
Gandalf and Aragorn lead the World of Men against Sauron's army to draw his gaze fro

In [253]:
df['Key_words'] = ""

for index, row in df.iterrows():
    plot = row['Plot']
        
    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words as keys and their scores as values
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column for the corresponding movie
    row['Key_words'] = list(key_words_dict_scores.keys())

# dropping the Plot column
#df.drop(columns = ['Plot'], inplace = True)

In [254]:
df['Key_words']

0      [two, imprisoned, men, bond, common, decency, finding, solace, acts, eventual, redemption, years...
1      [organized, crime, dynasty, transfers, control, clandestine, empire, reluctant, son, aging, patr...
2      [early, life, son, grip, family, crime, syndicate, 1920s, new, york, expands, career, michael, v...
3      [mysterious, past, greatest, psychological, chaos, people, menace, known, wreaks, havoc, fight, ...
4      [reconsider, miscarriage, colleagues, evidence, prevent, jury, holdout, attempts, forcing, justice]
                                                      ...                                                 
245                             [chronic, alcoholic, four, followed, day, drinking, bout, desperate, life]
246    [longtime, boyfriend, co, worker, troubled, waters, residential, treatment, facility, navigates,...
247               [book, wife, ace, reporter, ex, remarrying, newspaper, editor, uses, every, trick, keep]
248               [long, journey, men

In [192]:
#Remove punctuation and convert to lower
def genre(text):
  l= ''.join([word.lower() for word in text if word not in string.punctuation])
  return l


In [193]:
#remove delimiter and conver to lower
def director(text):
  t= ''.join(re.split('\W+', text.lower()))
  return t

In [194]:
#combine multiple names with space
def actors(text):
  l=''.join(re.split('\s+',text.lower()))
  l1=' '.join(re.split(",",l))
  return l1

In [197]:
def key(text):
  l=''.join(re.split('\s+',text))
  l1=' '.join(re.split(",",l))
  return l1

In [210]:
df['Genre1']= df['Genre'].apply(lambda x : genre(x))
df['Director1']= df['Director'].apply(lambda x : director(x))
df['Actors1']= df['Actors'].apply(lambda x : actors(x))
df['Key_words1']= df['Key_words'].apply(lambda x : ' '.join(x))


In [213]:
df['bag_of_words']= df['Genre1'].astype(str)+' '+ df['Director1'].astype(str)+ ' '+ df['Actors1'].astype(str)+ ' '+ df['Key_words1'].astype(str)

In [214]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])

# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [215]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices = pd.Series(df.index)

#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    # initializing the empty list of recommended movies
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

In [250]:
title= "Batman Begins"
j=recommendations(df[df['Title']==title].index[0], cosine_sim)
for i in j:
  print(df.iloc[i]["Title"])

The Dark Knight
The Dark Knight Rises
Spider-Man: Homecoming
Star Wars: Episode V - The Empire Strikes Back
The Prestige
Guardians of the Galaxy Vol. 2
Sin City
Baby Driver
Inception
Song of the Sea


The Dark Knight
The Dark Knight Rises
Spider-Man: Homecoming
Star Wars: Episode V - The Empire Strikes Back
The Prestige
Guardians of the Galaxy Vol. 2
Sin City
Baby Driver
Inception
Song of the Sea
